<a href="https://colab.research.google.com/github/KwonYN/TIL/blob/master/PYTHON/TENSORFLOW2/10_02_cnn_modern_7_kyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **실습 5-1 : Modern CNN**

## **Import Module**

In [0]:
%tensorflow_version 2.x

import tensorflow as tf
#import tensorflow.keras as keras
from tensorflow.keras import models, datasets
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPool2D, MaxPool1D
from tensorflow.keras.layers import Dropout, BatchNormalization

import numpy as np
import matplotlib.pyplot as plt

tf.__version__

TensorFlow 2.x selected.


'2.0.0'

## **DataSet**

## 2 `TensorFlow datasets` 사용하기

### 2.1 tf.keras.utils.get_file()   
# -- TF에서 제공되는 DataSet을 지정된 위치에서 읽어, colab에 압축 풀어 저장하기  
 -- DataSet Document https://www.tensorflow.org/datasets/catalog/overview 에서 URL 참조

In [0]:
import pathlib
data_dir = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)
print (data_dir)

228818944/228813984 [==============================] - 5s 0us/step
/root/.keras/datasets/flower_photos


### 2.2 저장된 이미지를 `ImageDataGenerator()`로 준비하기 

### 1.2 check images

In [0]:
# 지정 폴더 아래에 있는 모든 *.jpg 파일의 수
#  및 폴더명 목록을 리턴  
def check_dir(d_path):
  img_count = len(list(d_path.glob('*/*.jpg')))
  c_name = np.array([item.name for item in d_path.glob('*') if item.name != "LICENSE.txt"])
  return img_count, c_name, len(c_name)

In [0]:
# check_dir()로 폴더명과 이미지 숫자 확인
image_count, CLASS_NAMES, class_num = check_dir(data_dir)

print('image_count: {}\nclasses: {}'.format(image_count, CLASS_NAMES))

image_count: 3670
classes: ['roses' 'tulips' 'dandelion' 'sunflowers' 'daisy']


In [0]:
# input image size 지정 ***
im_size = 112

In [0]:
# image generator for unziped directory 
# The 1./255 is to convert from uint8 to float32 in range [0,1].
image_generator = (
    tf.keras.preprocessing.image.ImageDataGenerator(
        #width_shift_range=0.1,
        #height_shift_range=0.1,
        horizontal_flip=True, 
        vertical_flip=True,
        #rotation_range=20,
        #zoom_range=0.15,
        brightness_range=[0.8,1.2],
        validation_split=0.2,
        rescale=1./255))

# Batch size 지정 ***
batch_n = 128

# generate train dataset
train_data_gen = image_generator.flow_from_directory(
                      directory=str(data_dir),
                      batch_size=batch_n,
                      target_size=(im_size, im_size),
                      classes = list(CLASS_NAMES),
                      subset='training'
                      )

test_data_gen = image_generator.flow_from_directory(
                      directory=str(data_dir),
                      batch_size=batch_n,
                      target_size=(im_size, im_size),
                      classes = list(CLASS_NAMES),
                      subset='validation'
                      )


Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


## **Model**

### Define

### Model_A define: Dropout 사용 model

In [0]:
def model_cnn_basic():
  model = models.Sequential()
  # conv 1
  model.add(Conv2D(64,3,padding='same',activation='relu',input_shape=(im_size,im_size,3)))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 2
  model.add(Conv2D(128, 3, padding='same', activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 3
  model.add(Conv2D(256, 3, padding='same', activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # Dense layers
  model.add(Flatten(name='flatten'))
  model.add(Dense(1024, activation='relu', name='dense_1024'))
  model.add(Dense(len(CLASS_NAMES), activation='softmax', name='dense_10'))

  return model

model_basic = model_cnn_basic()

#model_basic.summary()

In [0]:
def model_cnn_dropout():
  model = models.Sequential()
  # conv 1
  model.add(Conv2D(64,3,padding='same',activation='relu',input_shape=(im_size,im_size,3)))
  model.add(Dropout(rate=0.25))                          # DO
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 2
  model.add(Conv2D(128, 3, padding='same', activation='relu'))
  model.add(Dropout(rate=0.25))                          # DO
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 3
  model.add(Conv2D(256, 3, padding='same', activation='relu'))
  model.add(Dropout(rate=0.25))                          # DO
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # dense layers
  model.add(Flatten(name='flatten'))
  model.add(Dense(1024, activation='relu', name='dense_1024'))
  model.add(Dense(len(CLASS_NAMES), activation='softmax', name='dense_10'))

  return model

model_DO = model_cnn_dropout()

#model_DO.summary()

### Model_B define: Batch Normalization 사용 model

In [0]:
def model_cnn_batchnormal():
  model = tf.keras.models.Sequential()
  # conv 1
  model.add(Conv2D(32,3,padding='same',activation='relu',input_shape=(im_size,im_size,3))) 
  model.add(BatchNormalization(momentum=0.85))             # BN
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 2
  model.add(Conv2D(64, 3, padding='same', activation='relu'))
  model.add(BatchNormalization(momentum=0.85))             # BN
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 3
  model.add(Conv2D(128, 3, padding='same', activation='relu')) 
  model.add(BatchNormalization(momentum=0.85))             # BN
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # conv 4
  model.add(Conv2D(128, 3, padding='same', activation='relu')) 
  model.add(BatchNormalization(momentum=0.85))             # BN
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  # dense layers
  model.add(Flatten(name='flatten'))
  model.add(Dense(1024, activation='relu', name='dense_1024'))
  model.add(Dense(len(CLASS_NAMES), activation='softmax', name='dense_10'))

  return model

model_BN=model_cnn_batchnormal()

#model_BN.summary()

In [0]:
def model_cnn_chal3():
  model = models.Sequential()

  # conv 1
  model.add(Conv2D(64,3,padding='same',activation='relu',input_shape=(im_size,im_size,3)))
  model.add(Conv2D(64, 3, padding='same', activation='relu'))
  model.add(BatchNormalization(momentum=0.85))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

  # conv 2
  model.add(Conv2D(128, 3, padding='same', activation='relu'))
  model.add(Conv2D(128, 3, padding='same', activation='relu'))
  model.add(BatchNormalization(momentum=0.85))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

  # conv 3
  model.add(Conv2D(256, 3, padding='same', activation='relu'))
  model.add(Conv2D(256, 3, padding='same', activation='relu'))
  model.add(BatchNormalization(momentum=0.85))
  model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))

  # conv 4
  model.add(Conv2D(256, 3, padding='same', activation='relu'))
  model.add(Conv2D(256, 3, padding='same', activation='relu'))
  model.add(BatchNormalization(momentum=0.85))
  model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))

  # conv 4
  model.add(Conv2D(64, 3, padding='same', activation='relu'))
  model.add(BatchNormalization(momentum=0.85))
  model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))

  # Dense layers
  model.add(Flatten(name='flatten'))
  model.add(Dense(len(CLASS_NAMES), activation='softmax', name='dense_10'))

  return model

model_chal3 = model_cnn_chal3()

#model_basic.summary()

### Compile

In [0]:
model_basic.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [0]:
model_DO.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [0]:
model_BN.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [0]:
model_chal3.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss='categorical_crossentropy',
              metrics=['acc'])

### Fit

In [0]:
# batch_n = 512 move to generator
epoch_n = 60

Epoch 20/20
60000/60000 [==============================] - 5s 80us/sample - loss: 0.0276 - accuracy: 0.9911
CPU times: user 1min 7s, sys: 30.2 s, total: 1min 38s   
Wall time: 1min 40
(@Notebook Setting/GPU)

In [0]:
# Challenge 3!!

%%time
# reset data generator
train_data_gen.reset()
test_data_gen.reset()

# fit_gen
history_chal3 = model_chal3.fit_generator(train_data_gen, 
                        epochs=epoch_n,
                        validation_data=test_data_gen
                        )

Epoch 1/60
23/23 [==============================] - 44s 2s/step - loss: 1.4630 - acc: 0.4811 - val_loss: 2.1332 - val_acc: 0.3037
Epoch 2/60
23/23 [==============================] - 27s 1s/step - loss: 1.0220 - acc: 0.5842 - val_loss: 1.0673 - val_acc: 0.5882
Epoch 3/60
23/23 [==============================] - 27s 1s/step - loss: 0.8717 - acc: 0.6560 - val_loss: 1.0455 - val_acc: 0.5896
Epoch 4/60
23/23 [==============================] - 27s 1s/step - loss: 0.7827 - acc: 0.6876 - val_loss: 1.1527 - val_acc: 0.6156
Epoch 5/60
23/23 [==============================] - 27s 1s/step - loss: 0.7281 - acc: 0.7179 - val_loss: 0.9139 - val_acc: 0.6539
Epoch 6/60
23/23 [==============================] - 27s 1s/step - loss: 0.7050 - acc: 0.7193 - val_loss: 0.9491 - val_acc: 0.6334
Epoch 7/60
23/23 [==============================] - 27s 1s/step - loss: 0.6789 - acc: 0.7339 - val_loss: 1.0671 - val_acc: 0.6101
Epoch 8/60
23/23 [==============================] - 27s 1s/step - loss: 0.6314 - acc: 0.75

In [0]:
epoch_list = [15, 30, 45, epoch_n]

acc_list = history_chal3.history['val_acc']

for i in range(4):
  print(np.max(acc_list[0:i]))

ValueError: ignored

In [0]:
%%time
# reset data generator
train_data_gen.reset()
test_data_gen.reset()

# fit_gen
history_BN = model_BN.fit_generator(train_data_gen, 
                        epochs=epoch_n,
                        validation_data=test_data_gen
                        )

In [0]:
%%time
# reset data generator
train_data_gen.reset()
test_data_gen.reset()

history_basic = model_basic.fit_generator(train_data_gen, 
                        epochs=epoch_n,
                        validation_data=test_data_gen)

In [0]:
%%time
# reset data generator
train_data_gen.reset()
test_data_gen.reset()

history_DO = model_DO.fit_generator(train_data_gen, 
                        epochs=epoch_n,
                        validation_data=test_data_gen
                        )

Epoch 20/20
60000/60000 [==============================] - 4s 73us/sample - loss: 2.1958e-04 - accuracy: 1.0000
CPU times: user 1min 1s, sys: 25.2 s, total: 1min 26s   
Wall time: 1min 27s (@Notebook Setting/GPU)

## **Analysis**

In [0]:
print(np.max(history_basic.history['val_acc']))
print(np.max(history_DO.history['val_acc']))
print(np.max(history_BN.history['val_acc']))

0.7116  
0.7257  
0.7268

### Plot

In [0]:
history_basic.history.keys()

In [0]:
loss = history_basic.history['loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.title('Validation Loss')
plt.semilogy(epochs, history_basic.history['val_loss'], 'b', label='CNN')
plt.semilogy(epochs, history_DO.history['val_loss'], 'r', label='CNN_DO')
plt.semilogy(epochs, history_BN.history['val_loss'], 'g', label='CNN_BN')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([0.0, 0.6])
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.title('Validation Accuray')
plt.semilogy(epochs, history_basic.history['val_acc'], 'b', label='CNN')
plt.semilogy(epochs, history_DO.history['val_acc'], 'r', label='CNN_DO')
plt.semilogy(epochs, history_BN.history['val_acc'], 'g', label='CNN_BN')
plt.grid(True)
plt.ylabel('Accuracy')
plt.ylim([0.5, 0.9])
plt.legend(loc='best')
plt.show()

## **실습 과제**

### 과제1 성능 개선을 위해서 다양한 실험이 필요해 보인다.
#### -- Layer / Feature map 숫자 변경, 
#### -- dropout / batch normalization 위치 및 숫자 변경,
#### -- batch size, epoch 변경을 통해 모델을 최적화 해 보자 
#### -- 조별로 최고의 성능을 달성한 모델의 구조와 주요 hyper-parameter 및 최고 val_acc를 기록하자 : 
